# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from scipy import stats as st
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
def display_data_summary(data):
    # Вывод первых 5 строк данных
    print("Первые 5 строк данных:")
    print(data.head())
    print()
    # Вывод общей информации о данных
    print("Информация о данных:")
    print(data.info())
    print()
    # Вывод основной статистической информации о данных
    print("Основная статистическая информация:")
    print(data.describe())

In [6]:
display_data_summary(data_1)

Первые 5 строк данных:
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Основная статистическая информация:
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143      

In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [10]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [11]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [12]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [13]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
data_3.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [15]:
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())
print(data_3.duplicated().sum())

0
0
0


In [16]:
data_1.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [17]:
data_2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [18]:
data_3.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [19]:
data_1['id'].unique()

array(['txEyH', '2acmU', '409Wp', ..., '3rnvd', '7kl59', '1CWhH'],
      dtype=object)

In [20]:
data_2['id'].unique()

array(['kBEdx', '62mP7', 'vyE1P', ..., '09gWa', 'rqwUm', 'relB0'],
      dtype=object)

In [21]:
data_3['id'].unique()

array(['fwXo0', 'WJtFt', 'ovLUW', ..., 'tKPY3', 'nmxp2', 'V9kWn'],
      dtype=object)

* Пропусков в данных не обнаружено
* Размер трёх выборок одинаковый
* Дубликатов в данных не обнаружено
* Данные представлены в одном мастштабе
* Все признаки являются численными

## Обучение и проверка модели

In [22]:
data_1_train, data_1_valid = train_test_split(data_1, test_size=0.25, random_state=12345)
print(data_1_train.shape)
data_1_valid.shape

(75000, 5)


(25000, 5)

Делим данные 1 региона на две выборки: обучающую (75%) и валидационную (25%).
В обучающие признаки данные из обучающей выборки, исключив ненужные столбцы. В целевой признак внесём столбец product.

In [23]:
features_train_1 = data_1_train.drop(['id', 'product'], axis=1)
target_train_1 = data_1_train['product']
features_valid_1 = data_1_valid.drop(['id', 'product'], axis=1)
target_valid_1 = data_1_valid['product']

Обучаем модель

In [24]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)

In [25]:
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1) ** 0.5
rmse_1

37.5794217150813

In [26]:
# вычисляем средний запас предсказанного сырья
product_pred_1 = predictions_valid_1.mean()
product_pred_1

92.59256778438035

In [27]:
data_2_train, data_2_valid = train_test_split(data_2, test_size = 0.25, random_state=12345)
print(data_2_train.shape)
data_2_valid.shape

(75000, 5)


(25000, 5)

In [28]:
features_train_2 = data_2_train.drop(['id', 'product'], axis=1)
target_train_2 = data_2_train['product']
features_valid_2 = data_2_valid.drop(['id', 'product'], axis=1)
target_valid_2 = data_2_valid['product']

In [29]:
model_2 = LinearRegression()
model_1.fit(features_train_2, target_train_2)
predictions_valid_2 = model_1.predict(features_valid_2)

In [30]:
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2) ** 0.5
rmse_2

0.893099286775617

In [31]:
rms = mean_squared_error(target_valid_1, predictions_valid_1, squared=False)
rms

37.5794217150813

In [32]:
product_pred_2 = predictions_valid_2.mean()
product_pred_2

68.728546895446

In [33]:
data_3_train, data_3_valid = train_test_split(data_3, test_size=0.25, random_state=12345)
print(data_3_train.shape)
data_3_valid.shape

(75000, 5)


(25000, 5)

In [34]:
features_train_3 = data_3_train.drop(['id', 'product'], axis=1)
target_train_3 = data_3_train['product']
features_valid_3 = data_3_valid.drop(['id', 'product'], axis=1)
target_valid_3 = data_3_valid['product']

In [35]:
model_3 = LinearRegression()
model_3.fit(features_train_3, target_train_3)
predictions_valid_3 = model_3.predict(features_valid_3)

In [36]:
rmse_3 = mean_squared_error(target_valid_3, predictions_valid_3) ** 0.5
rmse_3

40.02970873393434

In [37]:
product_pred_3 = predictions_valid_3.mean()
product_pred_3

94.96504596800489

In [38]:
def split_and_train(data):
    features = data.drop(["product","id"], axis=1)
    target = data["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=1)
    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    lr = LinearRegression(normalize=False)
    lr.fit(features_train, target_train)
    predictions = lr.predict(features_valid)
    predictions = pd.Series(predictions)
    rmse = (mean_squared_error(predictions, target_valid))**(0.5)
    product = sum(predictions) / len(predictions)
    
    print("RMSE: {0:.2f}".format(rmse))
    print("product: {0:.2f}".format(product))
    
    return (predictions, target_valid.reset_index(drop=True), rmse)

In [39]:
i = 1
for region in [data_1, data_2, data_3]:
    print(f"Region {i}:")
    
    split_and_train(region)
    
    i+=1

Region 1:
RMSE: 37.74
product: 92.49
Region 2:
RMSE: 0.89
product: 69.12
Region 3:
RMSE: 39.87
product: 94.96


In [40]:
table_1 = pd.DataFrame(index=['средний запас предсказанного сырья','RMSE модели'], columns=['1 регион','2 регион','3 регион'])
table_1['1 регион'] = product_pred_1.mean(), rmse_1
table_1['2 регион'] = product_pred_2.mean(), rmse_2
table_1['3 регион'] = product_pred_3.mean(), rmse_3
table_1

,1 регион,2 регион,3 регион
средний запас предсказанного сырья,92.592568,68.728547,94.965046
RMSE модели,37.579422,0.893099,40.029709


**Вывод**

Модели были обучены и протестированы на точность предсказания с использованием метрики RMSE, которая определяет среднеквадратичную ошибку в предсказаниях модели. Анализируя результаты, представленные в нашей сводной таблице, можно заметить, что значение RMSE во втором регионе существенно отличается от двух других. Это объясняется тем, что модель была обучена на практически равных данных по объему, и поэтому ошибка в предсказаниях этой модели не может сильно варьироваться

## Подготовка к расчёту прибыли

При разведке региона исследуюем 500 точек, из которых с помощью машинного обучения выберем 200 лучших для разработки.
Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс.
рублей, поскольку объём указан в тысячах баррелей.
После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%.

In [41]:
budget = 10_000_000_000

In [42]:
total_count = 500

In [43]:
count_of_wells = 200

In [44]:
price_one_well = budget / count_of_wells 

In [45]:
price_one_thousand_barrels = 450_000

In [46]:
samples_number = 1000

In [47]:
n = budget / price_one_thousand_barrels
print("Точка безубыточности: {0:.0f} тыс. баррелей".format(n))

Точка безубыточности: 22222 тыс. баррелей


In [48]:
print("Средний объем нефти в скважине для безубыточного функционирования:", np.ceil(n/200))

Средний объем нефти в скважине для безубыточного функционирования: 112.0


При существующем уровне цен в 450 тыс.руб., безубыточная добыча начинается со скважин с запасом в 112 тыс.баррелей. По предсказаниям в среднем такого объема нет ни в одном регионе.

## Расчёт прибыли и рисков 

In [49]:
target_valid_1 = pd.Series(target_valid_1).reset_index(drop=True)
predictions_valid_1 = pd.Series(predictions_valid_1).reset_index(drop=True)

target_valid_2 = pd.Series(target_valid_2).reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2).reset_index(drop=True)

target_valid_3 = pd.Series(target_valid_3).reset_index(drop=True)
predictions_valid_3 = pd.Series(predictions_valid_3).reset_index(drop=True)

In [50]:
def revenue(target, predictions, count_of_wells):
    preds_sorted = predictions.sort_values(ascending=False)
    selected = target[preds_sorted.index][:count_of_wells]
    return price_one_thousand_barrels * selected.sum() - budget

In [51]:
target = [target_valid_1, target_valid_2, target_valid_3]
predictions = [predictions_valid_1, predictions_valid_2, predictions_valid_3]
regions = ['Первый регион', 'Второй регион', 'Третий регион']

state = np.random.RandomState(12345)

for m in range(len(target)):
    values = []
    for i in range(samples_number):
        target_subsample = target[m].sample(n=total_count, replace=True, random_state=state)
        predictions_subsample = predictions[m].loc[target_subsample.index]
        values.append(revenue(target_subsample.reset_index(drop=True), predictions_subsample.reset_index(drop=True), count_of_wells))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    mean = values.mean()
    
    #loss = sum(values < 0) / values.count() * 100
    risk = (values < 0).mean()
    
    print(regions[m])
    print("Средняя возможная прибыль:", mean)
    print("95%-доверительный интервал:", "от", lower, "до", upper)
    print("Вероятность убытков:", risk, "%")

Первый регион
Средняя возможная прибыль: 396164984.8023711
95%-доверительный интервал: от -111215545.89049526 до 909766941.5534226
Вероятность убытков: 0.069 %
Второй регион
Средняя возможная прибыль: 461155817.2772397
95%-доверительный интервал: от 78050810.7517417 до 862952060.2637234
Вероятность убытков: 0.007 %
Третий регион
Средняя возможная прибыль: 392950475.17060447
95%-доверительный интервал: от -112227625.37857565 до 934562914.5511636
Вероятность убытков: 0.065 %


**Вывод**

Согласно распределению среднего значения прибыли с региона, самым прибыльным оказался 2, средняя прибыль в этом регионе составит 461 млн.рублей. Остальные регионы отбор по минимально допустимому порогу убытка в 2,5% не прошли. Таким образом не смотря на сравнительно меньшие запасы сырья, по сравнению с 1 регионом, средняя доходность со скважин в 2 регионе прогнозируется выше.

Мы исследовали данные по запасам в скважинах трёх регионов, в каждом из которых были данные о запасах сырья по 10 тыс.скважинам. Мы обнаружили, что средний запас ресурсов по регионам недостаточен даже для окупаемости вложений на разработку и для того, чтобы иметь уверенность в перспективе разработки по регионам, мы обучили модель и выполнили предсказания при помощи, а так же применили технологию bootstrap с разделением каждого предсказанного значения прибыли по регионам на 1000 выборок.

По результату полученных данных мы определили доверительный интревал получения прибыли в 95%, ограничив вероятность убытка величиной менее 2,5%. И на основе этих данных смогли выбрать более перспективный регион ( им оказался регион 2) для разработки 200 скважин.

## Чек-лист готовности проекта


- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован